### Leveraging Parquet File Format efficiency & Enforcing better Schemas in our data

- **Start Spark Application** <br>
This is just to turn the spark application on and retrieve an Application ID

In [ ]:
spark

- **Define the HDFS Data Directories that will host parquet files**
- **NOTE : As you are working in pairs on the same cluster, if you want both to work simultaneously, you should create each create your parquet folders with specific names and modify the botebook accordingly**
  


`hdfs dfs -mkdir /user/root/data/BOTS/PARQUET`<br>
`hdfs dfs -mkdir /user/root/data/BOTS_REF/PARQUET`<br>

In [ ]:
BOTS_DIR_HDFS_CSV = "hdfs:///user/root/data/BOTS/CSV"
BOTS_DIR_HDFS_PQT = "hdfs:///user/root/data/BOTS/PARQUET"

##### Timing Utility

In [ ]:
from time import time
from datetime import timedelta

class T():
    def __enter__(self):
        self.start = time()
    def __exit__(self, type, value, traceback):
        self.end = time()
        elapsed = self.end - self.start
        print(str(timedelta(seconds=elapsed)))

##### Usage : 

```
with T():
    //instructions you want to time
```
Note : %%time magic command does not work with this version of pyspark kernel.

<hr style="border:1px solid blue"></hr>

#### Attesting Parquet Performance


- **Load a Dataframe :**<br>


In [ ]:
with T():
    df_csv = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(BOTS_DIR_HDFS_CSV)

In [ ]:
with T():
    print(df_csv.count())

- **Print CSV DataFrame Schema**<br>

In [ ]:
df_csv.printSchema()

- **Write the Dataframe to Parquet Format**
- Use the right dedicated subdirectory and add a sensible filename with a .parquet extension

In [ ]:
BOTS_FILE_HDFS_PQT = BOTS_DIR_HDFS_PQT +"/"+"BOTS_2019.parquet"

In [ ]:
df_csv.write.parquet(BOTS_FILE_HDFS_PQT)

- Now Explore Parquet Files on HDFS using your SSH Termina

`hdfs dfs -ls /user/root/data/BOTS/PARQUET`  
`hdfs dfs -ls /user/root/data/BOTS/PARQUET/BOTS_2019.parquet`

- What do you observe in the folder ?
- How is a parquet File structured (link that to the course) ?

- **Read the Dataframe back from the Parquet File**
- Compare reading time with reading from CSV Format
- What do you observe ?

In [ ]:
with T():
    df_parquet = spark.read.format("parquet").load(BOTS_FILE_HDFS_PQT)

In [ ]:
with T():
    print(df_parquet.count())

- **Print Parquet DataFrame Schema**<br>

In [ ]:
df_parquet.printSchema()

- What do you notice ?
- Where in your opinion were schema information stored ?

### Assessing performance : 
Try operations form the previous notebook (distinct(), or try any other transformation like filter, groupBy, or else)  
on both `df_csv` and `df_parquet`, compare timings, and share your thoughts

In [ ]:
## TODO : FILL IN


<hr style="border:1px solid blue"></hr>

### Defining & applying better schemas to your dataframe :

- Explore your dataframe and the inferrred schema and try to improve the schema  

    * A strong schema makes for better performance and better reliability
    * Below is an example with a movie dataset that we will be using later
    * Try to apply that to your BOTS dataframe
    * save back to parquet
    * load again from parquet without any inference
    * print your schema 
    * check that your schema was save correctly to parquet
    


Example with a Movies Dataset 

````
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
````
You should add every type you need to this import
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types


````
moviesStruct = 
   [StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True)]

moviesSchema = StructType(moviesStruct)
````

- You will face a small practical problem though : You have a very large number of columns

- A Handy alternative is to use datatype strings : 

- The following CELLS are EXAMPLE Cells  
- You need to apply them to you specific dataset (it time permits)


In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

data2 = [("James","Smith","36636","M",3000),
    ("Michael","Rose","40288","M",4000),
    ("Robert","Williams","42114","M",4000),
    ("Maria","Jones","39192","F",4000),
    ("Jen","Brown","","F",2000)
  ]

In [ ]:
dff.printSchema()

In [ ]:
dff = spark.createDataFrame(data=data2)

In [ ]:

# we can use a datatype string - This is not much documented actually, 
# but way easier, less verbose, and more straightforward than using structypes

ddlSchema = ("FIRSTNAME STRING, NAME STRING, ZIPCODE STRING, GENDER STRING, SALARY INT")


In [ ]:
dff = spark.createDataFrame(data=data2,schema=ddlSchema)

In [ ]:
dff.printSchema()

In [ ]:
# TODO : FILL IN
# APPLY TO YOUR PARQUET DATASET
# IF TIME PERMITS